# 🎨 Data Designer 101: Seeding Synthetic Data Generation with an External Dataset

[Click here](https://raw.githubusercontent.com/NVIDIA-NeMo/DataDesigner/refs/heads/main/docs/notebooks/3-seeding-with-a-dataset.ipynb) to download this notebook to your computer.
#### 📚 What you'll learn

In this notebook, we will demonstrate how to seed synthetic data generation in Data Designer with an external dataset.

If this is your first time using Data Designer, we recommend starting with the [first notebook](/notebooks/1-the-basics/) in this 101 series.

### 📦 Import the essentials

- The `essentials` module provides quick access to the most commonly used objects.


In [1]:
from data_designer.essentials import (
    DataDesigner,
    DataDesignerConfigBuilder,
    InferenceParameters,
    ModelConfig,
    SeedConfig,
)

### ⚙️ Initialize the Data Designer interface

- `DataDesigner` is the main object that is used to interface with the library.


In [2]:
data_designer_client = DataDesigner()

[16:33:56] [INFO] ♻️ Using default model providers from '/Users/amanoel/.data-designer/model_providers.yaml'


### 🎛️ Define model configurations

- Each `ModelConfig` defines a model that can be used during the generation process.

- The "model alias" is used to reference the model in the Data Designer config (as we will see below).

- The "model provider" is the external service that hosts the model (see the [model config](/models/default-model-settings.md) docs for more details).

- By default, we use [build.nvidia.com](https://build.nvidia.com/models) as the model provider.


In [3]:
# This name is set in the model provider configuration.
MODEL_PROVIDER = "nvidia"

# The model ID is from build.nvidia.com.
MODEL_ID = "nvidia/nvidia-nemotron-nano-9b-v2"

# We choose this alias to be descriptive for our use case.
MODEL_ALIAS = "nemotron-nano-v2"

# This sets reasoning to False for the nemotron-nano-v2 model.
SYSTEM_PROMPT = "/no_think"

model_configs = [
    ModelConfig(
        alias=MODEL_ALIAS,
        model=MODEL_ID,
        provider=MODEL_PROVIDER,
        inference_parameters=InferenceParameters(
            temperature=0.5,
            top_p=1.0,
            max_tokens=1024,
        ),
    )
]

### 🏗️ Initialize the Data Designer Config Builder

- The Data Designer config defines the dataset schema and generation process.

- The config builder provides an intuitive interface for building this configuration.

- The list of model configs is provided to the builder at initialization.


In [4]:
config_builder = DataDesignerConfigBuilder(model_configs=model_configs)

## 🏥 Prepare a seed dataset

- For this notebook, we'll create a synthetic dataset of patient notes.

- We will _seed_ the generation process with a [symptom-to-diagnosis dataset](https://huggingface.co/datasets/gretelai/symptom_to_diagnosis).

- We already have the dataset downloaded in the [data](../data) directory of this repository.

<br>

> 🌱 **Why use a seed dataset?**
>
> - Seed datasets let you steer the generation process by providing context that is specific to your use case.
>
> - Seed datasets are also an excellent way to inject real-world diversity into your synthetic data.
>
> - During generation, prompt templates can reference any of the seed dataset fields.


In [5]:
# Download sample dataset from Github
import urllib.request

url = "https://raw.githubusercontent.com/NVIDIA/GenerativeAIExamples/refs/heads/main/nemo/NeMo-Data-Designer/data/gretelai_symptom_to_diagnosis.csv"
local_filename, headers = urllib.request.urlretrieve(url, "gretelai_symptom_to_diagnosis.csv")

seed_dataset = SeedConfig(dataset=local_filename)

# Pass the reference to the config builder for use during generation.
config_builder.with_seed_dataset(seed_dataset)

DataDesignerConfigBuilder(
    seed_dataset: 'gretelai_symptom_to_diagnosis.csv'
    seed_dataset_columns: ['diagnosis', 'patient_summary']
)

## 🎨 Designing our synthetic patient notes dataset

- Here we use `add_column` with keyword arguments (rather than imported config objects).

- Generally, we recommend using concrete objects, but this is a convenient shorthand.

- **Note**: The prompt template can reference fields from our seed dataset:
  - `{{ diagnosis }}` - the medical diagnosis from the seed data
  - `{{ patient_summary }}` - the symptom description from the seed data


In [6]:
config_builder.add_column(
    name="patient_sampler",
    column_type="sampler",
    sampler_type="person_from_faker",
    drop=False,
)

config_builder.add_column(
    name="doctor_sampler",
    column_type="sampler",
    sampler_type="person_from_faker",
    drop=False,
)

config_builder.add_column(
    name="patient_id",
    column_type="sampler",
    sampler_type="uuid",
    params={
        "prefix": "PT-",
        "short_form": True,
        "uppercase": True,
    },
)

config_builder.add_column(
    name="first_name",
    column_type="expression",
    expr="{{ patient_sampler.first_name}}",
)

config_builder.add_column(
    name="last_name",
    column_type="expression",
    expr="{{ patient_sampler.last_name }}",
)


config_builder.add_column(
    name="dob",
    column_type="expression",
    expr="{{ patient_sampler.birth_date }}",
)

config_builder.add_column(
    name="symptom_onset_date",
    column_type="sampler",
    sampler_type="datetime",
    params={"start": "2024-01-01", "end": "2024-12-31"},
)

config_builder.add_column(
    name="date_of_visit",
    column_type="sampler",
    sampler_type="timedelta",
    params={"dt_min": 1, "dt_max": 30, "reference_column_name": "symptom_onset_date"},
)

config_builder.add_column(
    name="physician",
    column_type="expression",
    expr="Dr. {{ doctor_sampler.last_name }}",
)

config_builder.add_column(
    name="physician_notes",
    column_type="llm-text",
    prompt="""\
You are a primary-care physician who just had an appointment with {{ first_name }} {{ last_name }},
who has been struggling with symptoms from {{ diagnosis }} since {{ symptom_onset_date }}.
The date of today's visit is {{ date_of_visit }}.

{{ patient_summary }}

Write careful notes about your visit with {{ first_name }},
as Dr. {{ doctor_sampler.first_name }} {{ doctor_sampler.last_name }}.

Format the notes as a busy doctor might.
""",
    model_alias=MODEL_ALIAS,
    system_prompt=SYSTEM_PROMPT,
)

config_builder.validate()

[16:33:57] [INFO] ✅ Validation passed


DataDesignerConfigBuilder(
    seed_dataset: 'gretelai_symptom_to_diagnosis.csv'
    seed_dataset_columns: ['diagnosis', 'patient_summary']
    sampler_columns: [
        "patient_sampler",
        "doctor_sampler",
        "patient_id",
        "symptom_onset_date",
        "date_of_visit"
    ]
    llm_text_columns: ['physician_notes']
    expression_columns: [
        "first_name",
        "last_name",
        "dob",
        "physician"
    ]
)

### 🔁 Iteration is key – preview the dataset!

1. Use the `preview` method to generate a sample of records quickly.

2. Inspect the results for quality and format issues.

3. Adjust column configurations, prompts, or parameters as needed.

4. Re-run the preview until satisfied.


In [7]:
preview = data_designer_client.preview(config_builder)

[16:33:57] [INFO] 📸 Preview generation in progress
[16:33:57] [INFO] ✅ Validation passed
[16:33:57] [INFO] ⛓️ Sorting column configs into a Directed Acyclic Graph
[16:33:57] [INFO] 🩺 Running health checks for models...
[16:33:57] [INFO]   |-- 👀 Checking 'nvidia/nvidia-nemotron-nano-9b-v2' in provider named 'nvidia' for model alias 'nemotron-nano-v2'...
[16:33:58] [INFO]   |-- ✅ Passed!
[16:33:58] [INFO] 🌱 Sampling 10 records from seed dataset
[16:33:58] [INFO]   |-- seed dataset size: 820 records
[16:33:58] [INFO]   |-- sampling strategy: ordered
[16:33:58] [INFO] 🎲 Preparing samplers to generate 10 records across 5 columns
[16:33:58] [INFO] (💾 + 💾) Concatenating 2 datasets
[16:33:58] [INFO] 🧩 Generating column `first_name` from expression
[16:33:58] [INFO] 🧩 Generating column `last_name` from expression
[16:33:58] [INFO] 🧩 Generating column `dob` from expression
[16:33:58] [INFO] 🧩 Generating column `physician` from expression
[16:33:58] [INFO] 📝 Preparing llm-text column generation
[

In [8]:
# Run this cell multiple times to cycle through the 10 preview records.
preview.display_sample_record()

                                                   Seed Columns                                                    
┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Name            ┃ Value                                                                                         ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ diagnosis       │ cervical spondylosis                                                                          │
├─────────────────┼───────────────────────────────────────────────────────────────────────────────────────────────┤
│ patient_summary │ I've been having a lot of pain in my neck and back. I've also been having trouble with my     │
│                 │ balance and coordination. I've been coughing a lot and my limbs feel weak.                    │
└─────────────────┴───────────────────────────────────────────────────────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                 Generated Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Name               ┃ Value                                                                                      ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ patient_sampler    │ {                                                                                          │
│                    │     'uuid': '68174e80-5e4b-4e5c-b68f-91bec63746ef',                                        │
│                    │     'locale': 'en_US',                                                                     │
│                    │     'first_name': 'Wendy',                                                                 │
│                    │     'last_name': 'Guzman',                                                                 │
│                    │     'middle_name': None,                                                                   │
│                    │     'sex': 'Female',                                                                       │
│                    │     'street_number': '7569',                                                               │
│                    │     'street_name': 'Beth Ports',                                                           │
│                    │     'city': 'West Lauren',                                                                 │
│                    │     'state': 'Arizona',                                                                    │
│                    │     'postcode': '62783',                                                                   │
│                    │     'age': 99,                                                                             │
│                    │     'birth_date': '1926-06-11',                                                            │
│                    │     'country': 'Armenia',                                                                  │
│                    │     'marital_status': 'separated',                                                         │
│                    │     'education_level': 'secondary_education',                                              │
│                    │     'unit': '',                                                                            │
│                    │     'occupation': 'Engineer, materials',                                                   │
│                    │     'phone_number': '882-478-7638

In [9]:
# The preview dataset is available as a pandas DataFrame.
preview.dataset

,diagnosis,patient_summary,patient_sampler,doctor_sampler,patient_id,symptom_onset_date,date_of_visit,first_name,last_name,dob,physician,physician_notes
0,cervical spondylosis,I've been having a lot of pain in my neck and ...,{'uuid': '68174e80-5e4b-4e5c-b68f-91bec63746ef...,{'uuid': '6b0cc640-5322-4068-8e62-408648f2a01a...,PT-CB852B6C,2024-02-20,2024-02-21,Wendy,Guzman,1926-06-11,Dr. Aguilar,**Dr. Katherine Aguilar** \n**Primary Care Ph...
1,impetigo,I have a rash on my face that is getting worse...,{'uuid': '4e98c43b-8f7d-4bb4-8bd7-a7eafa0b556d...,{'uuid': '633df172-baa5-47bc-8e3b-d0b588542cdb...,PT-CF78B0FA,2024-09-18,2024-09-19,Amanda,Smith,1994-09-27,Dr. Marsh,"**Dr. Sherri Marsh, MD** \n**Primary Care Phy..."
2,urinary tract infection,I have been urinating blood. I sometimes feel ...,{'uuid': 'c1af3224-3584-4e40-a995-3460f958848d...,{'uuid': 'c59bda90-b568-4694-bf95-8fda720d6e79...,PT-12A85CCD,2024-10-12,2024-10-24,Timothy,Simmons,1917-04-30,Dr. Carpenter,**Dr. Elizabeth Carpenter** \n**Primary Care ...
3,arthritis,I have been having trouble with my muscles and...,{'uuid': '0ba463b6-0b83-43b7-b689-78b6d25b2f48...,{'uuid': '53f687e8-a9bd-4900-94ed-c9f23dfd0a07...,PT-FFEF37D6,2024-08-30,2024-09-05,Patricia,White,1984-11-27,Dr. Oliver,**Dr. Maria Oliver** \n**Primary Care Physici...
4,dengue,I have been feeling really sick. My body hurts...,{'uuid': 'f3ec1689-8b90-495c-b41a-fc64563a088c...,{'uuid': '1e6ce90f-0907-4843-acad-7fc6325a4b5f...,PT-DC0CCE6C,2024-12-19,2024-12-28,Leah,Weaver,1996-06-28,Dr. Wilkins,**Dr. Christopher Wilkins** \n**Primary Care ...
5,common cold,I've been feeling really run down and weak. My...,{'uuid': '99905039-a297-4539-8541-162c9bc0b944...,{'uuid': '9824c4af-77bd-41ef-b3de-96bcef1a3afd...,PT-79573082,2024-09-19,2024-09-24,Andrea,Morrison,1944-07-30,Dr. Williams,"**Dr. Tristan Williams, Primary Care Physician..."
6,dengue,I have rashes all over my body. I also have a ...,{'uuid': '94f729fc-c6f5-4f79-9930-cbe5b11591a5...,{'uuid': 'f95cc47e-19c6-410e-9432-22bd7b3b264f...,PT-CB395BC7,2024-04-27,2024-05-15,Holly,Jackson,1939-07-24,Dr. Hoffman,**Dr. Ashley Hoffman** \n**Primary Care Physi...
7,dengue,I have been feeling nauseous and have a consta...,{'uuid': '32e44e19-e6da-4696-a7fd-d78ab2d31bd8...,{'uuid': '629f6af3-f78c-4bf6-b253-c2b253d45813...,PT-F673CE25,2024-09-27,2024-09-29,Lance,Henderson,1981-11-17,Dr. Kennedy,**Dr. Kristen Kennedy** \n**Primary Care Phys...
8,impetigo,I have a rash around my nose that is red and i...,{'uuid': 'e61cefa1-254a-414b-baea-f9fa6696a854...,{'uuid': 'cbec5207-bd43-4c5c-b8b6-5129065eae6f...,PT-288E96A3,2024-07-14,2024-08-03,Megan,Pena,1973-09-30,Dr. Moore,**Dr. Pamela Moore** \n**Primary Care Physici...
9,drug reaction,I have a rash on my chest and back and it itch...,{'uuid': 'c893e5ec-fa33-4acd-83de-2724eadab37e...,{'uuid': 'eaf0d824-e65a-4e22-be8c-325f3d25bff7...,PT-1DE48AF1,2024-02-14,2024-03-05,Melvin,Thomas,1954-07-15,Dr. Leonard,"**Dr. Blake Leonard, Primary Care Physician** ..."


### 📊 Analyze the generated data

- Data Designer automatically generates a basic statistical analysis of the generated data.

- This analysis is available via the `analysis` property of generation result objects.


In [10]:
# Print the analysis as a table.
preview.analysis.to_report()

──────────────────────────────────────── 🎨 Data Designer Dataset Profile ─────────────────────────────────────────

                                                                                                                   
                                                 Dataset Overview                                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ number of records               ┃ number of columns               ┃ percent complete records                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 10                              │ 12                              │ 100.0%                                      │
└─────────────────────────────────┴─────────────────────────────────┴─────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                              🌱 Seed-Dataset Columns                                              
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                          ┃               data type ┃                           number unique values ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ diagnosis                            │                  string │                                      7 (70.0%) │
├──────────────────────────────────────┼─────────────────────────┼────────────────────────────────────────────────┤
│ patient_summary                      │                  string │                                    10 (100.0%) │
└──────────────────────────────────────┴─────────────────────────┴────────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                🎲 Sampler Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                   ┃       data type ┃             number unique values ┃               sampler type ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ patient_sampler               │            dict │                      10 (100.0%) │          person_from_faker │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ doctor_sampler                │            dict │                      10 (100.0%) │          person_from_faker │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ patient_id                    │          string │                      10 (100.0%) │                       uuid │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ symptom_onset_date            │          string │                      10 (100.0%) │                   datetime │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ date_of_visit                 │          string │                      10 (100.0%) │                  timedelta │
└───────────────────────────────┴─────────────────┴──────────────────────────────────┴────────────────────────────┘
                                                          

### 🆙 Scale up!

- Happy with your preview data?

- Use the `create` method to submit larger Data Designer generation jobs.


In [11]:
job_results = data_designer_client.create(config_builder, num_records=20)

[16:34:21] [INFO] 🎨 Creating Data Designer dataset
[16:34:21] [INFO] ✅ Validation passed
[16:34:21] [INFO] ⛓️ Sorting column configs into a Directed Acyclic Graph
[16:34:21] [INFO] 🩺 Running health checks for models...
[16:34:21] [INFO]   |-- 👀 Checking 'nvidia/nvidia-nemotron-nano-9b-v2' in provider named 'nvidia' for model alias 'nemotron-nano-v2'...
[16:34:22] [INFO]   |-- ✅ Passed!
[16:34:22] [INFO] ⏳ Processing batch 1 of 1
[16:34:22] [INFO] 🌱 Sampling 20 records from seed dataset
[16:34:22] [INFO]   |-- seed dataset size: 820 records
[16:34:22] [INFO]   |-- sampling strategy: ordered
[16:34:22] [INFO] 🎲 Preparing samplers to generate 20 records across 5 columns
[16:34:22] [INFO] (💾 + 💾) Concatenating 2 datasets
[16:34:22] [INFO] 🧩 Generating column `first_name` from expression
[16:34:22] [INFO] 🧩 Generating column `last_name` from expression
[16:34:22] [INFO] 🧩 Generating column `dob` from expression
[16:34:22] [INFO] 🧩 Generating column `physician` from expression
[16:34:22] [IN

In [12]:
# Load the generated dataset as a pandas DataFrame.
dataset = job_results.load_dataset()

dataset.head()

,diagnosis,patient_summary,patient_sampler,doctor_sampler,patient_id,symptom_onset_date,date_of_visit,first_name,last_name,dob,physician,physician_notes
0,cervical spondylosis,I've been having a lot of pain in my neck and ...,"{'age': 77, 'bachelors_field': 'no_degree', 'b...","{'age': 80, 'bachelors_field': 'no_degree', 'b...",PT-9B5AA9F0,2024-11-12,2024-11-21,Joseph,Mcconnell,1948-09-04,Dr. Odonnell,"**Dr. Lori Odonnell, MD** **Primary Care Phy..."
1,impetigo,I have a rash on my face that is getting worse...,"{'age': 18, 'bachelors_field': 'stem', 'birth_...","{'age': 28, 'bachelors_field': 'no_degree', 'b...",PT-879882E0,2024-08-01,2024-08-30,Ann,Thomas,2007-09-30,Dr. Thomas,**Dr. Melissa Thomas** **Primary Care Physic...
2,urinary tract infection,I have been urinating blood. I sometimes feel ...,"{'age': 35, 'bachelors_field': 'no_degree', 'b...","{'age': 79, 'bachelors_field': 'education', 'b...",PT-C8FAA660,2024-11-09,2024-11-24,Dalton,Phillips,1990-01-31,Dr. Johns,**Dr. Samantha Johns** **Primary Care Physic...
3,arthritis,I have been having trouble with my muscles and...,"{'age': 63, 'bachelors_field': 'stem_related',...","{'age': 110, 'bachelors_field': 'no_degree', '...",PT-ED015BD6,2024-06-08,2024-06-30,William,Price,1962-09-19,Dr. Smith,"**Dr. Leah Smith, Primary Care Physician** *..."
4,dengue,I have been feeling really sick. My body hurts...,"{'age': 94, 'bachelors_field': 'no_degree', 'b...","{'age': 29, 'bachelors_field': 'no_degree', 'b...",PT-0F6852CC,2024-11-09,2024-12-06,Ashley,Perez,1931-07-18,Dr. Koch,**Dr. Michael Koch** **Primary Care Physicia...


In [13]:
# Load the analysis results into memory.
analysis = job_results.load_analysis()

analysis.to_report()

──────────────────────────────────────── 🎨 Data Designer Dataset Profile ─────────────────────────────────────────

                                                                                                                   
                                                 Dataset Overview                                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ number of records               ┃ number of columns               ┃ percent complete records                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 20                              │ 12                              │ 100.0%                                      │
└─────────────────────────────────┴─────────────────────────────────┴─────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                              🌱 Seed-Dataset Columns                                              
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                          ┃               data type ┃                           number unique values ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ diagnosis                            │                  string │                                     12 (60.0%) │
├──────────────────────────────────────┼─────────────────────────┼────────────────────────────────────────────────┤
│ patient_summary                      │                  string │                                    20 (100.0%) │
└──────────────────────────────────────┴─────────────────────────┴────────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                🎲 Sampler Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                   ┃       data type ┃             number unique values ┃               sampler type ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ patient_sampler               │            dict │                      20 (100.0%) │          person_from_faker │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ doctor_sampler                │            dict │                      20 (100.0%) │          person_from_faker │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ patient_id                    │          string │                      20 (100.0%) │                       uuid │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ symptom_onset_date            │          string │                       19 (95.0%) │                   datetime │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ date_of_visit                 │          string │                       19 (95.0%) │                  timedelta │
└───────────────────────────────┴─────────────────┴──────────────────────────────────┴────────────────────────────┘
                                                          

## ⏭️ Next Steps

Use Data Designer to generate synthetic data for your specific use case!
